In [1]:
"""
- simpy에서 세팅한 시뮬레이션 월드는 프로세스를 중심으로 구현됩니다. 
- 동시에 여러 개의 프로세스를 돌릴 수 있는데(예를 들어 프로세스=차, 여러 개의 차가 드라이빙되고 있음)
- 이 때, 프로세스 간의 인터랙션을 어떻게 구현할 수 있는가? 
    - 일단 여기서는 interrupting/ waiting for a process 라고 구분해서 말하고 있음 

- 프로세스와 subprocess로 구분해서 볼 수도 있을 듯. 
"""
import simpy 
import numpy as np 

## 물론 이 아래 부분을 클래스로 구현을 해도 좋지만 일단은 이해를 위해서 다 함수로 표현함 
def subsubprocess(env):
    ## process의 개별 activity는 subprocess로 구성되어 있습니다. 
    print('        subsubprocess start at {:6.2f}'.format(env.now))
    for i in range(0, 2):
        execution_time = np.random.triangular(left=1, right=2, mode=1)
        yield env.timeout(execution_time)
    print('        subsubprocess over  at {:6.2f}'.format(env.now))
def subprocess(env):
    ## process의 개별 activity는 subprocess로 구성되어 있습니다. 
    print('    subprocess start at {:6.2f}'.format(env.now))
    for i in range(0, 2):
        yield env.process(subsubprocess(env))
    print('    subprocess over  at {:6.2f}'.format(env.now))
    
def process(env, activity_lst):
    while True:
        for act in activity_lst:
            print("start {} at {:6.2f}".format(act, env.now))
            execution_time = np.random.triangular(left=3, right=10, mode=6)
            ## 모든 activity는 subprocess라고 생각한다.
            ## subprocess(env)가 종료되어야 다음 스텝으로 넘어감
            ## 즉 일종의 waiting for other process를 구현했다고 보면 됨 
            yield env.process(subprocess(env))
            ##############
            print("end   {} at {:6.2f}".format(act, env.now))
            transfer_time = np.random.triangular(left=1, right=3, mode=2)
            yield env.timeout(transfer_time)
        print('process instance ends')
        print('#'*30)
        return None
###########
env = simpy.Environment()
process1 = process(env, ["act_{}".format(i) for i in range(0, 3)])
env.process(process1)
env.run(50)

start act_0 at   0.00
    subprocess start at   0.00
        subsubprocess start at   0.00
        subsubprocess over  at   2.77
        subsubprocess start at   2.77
        subsubprocess over  at   5.70
    subprocess over  at   5.70
end   act_0 at   5.70
start act_1 at   7.91
    subprocess start at   7.91
        subsubprocess start at   7.91
        subsubprocess over  at  10.34
        subsubprocess start at  10.34
        subsubprocess over  at  12.78
    subprocess over  at  12.78
end   act_1 at  12.78
start act_2 at  15.53
    subprocess start at  15.53
        subsubprocess start at  15.53
        subsubprocess over  at  18.20
        subsubprocess start at  18.20
        subsubprocess over  at  20.39
    subprocess over  at  20.39
end   act_2 at  20.39
process instance ends
##############################


In [59]:
import simpy 
import numpy as np 

## 현재 env에 있는 모든 process를 쭉 접근할 수 있는 방법이 없음.
## 따라서, 매번 프로세스를 따로 리스트의 형태로 저장해주는 것이 필요함. 
current_ps = []

def clock(env, i, tick):
    ## generator에 interrupt 가 발생했을 때 종료하는 조건을 넣어주어야 함 
    while True:
        try: 
            yield env.timeout(tick)
            print('clock {} ticks at {}'.format(i, env.now))
        except simpy.Interrupt:
            print("## the clock {} was interrupted".format(i))
            return None
            
def stop_any_process(env):
    ## 2초마다 한번씩 현재 process 중 아무거나 종료시키는 generator
    ## 남아있는 clock이 없을때의 조건도 만들어줌. 
    while True:
        try:
            yield env.timeout(2)
            r = np.random.randint(0, len(current_ps))
            current_ps[r].interrupt()
            current_ps.remove(current_ps[r])
        except:
            print("#"*20)
            print("all process was interrupted at {}".format(env.now))
            return None
        
## environment setting
env = simpy.Environment()

## 6 개의 중간에 멈출 수 있는 clock을 만들어서 집어넣음
for i in range(0, 5):
    p = env.process(clock(env, i, 2))
    ## 새롭게 만들어진 프로세스에 대해서 외부에서 접근 방법이 없으므로, 따로 저장해두어야 함
    current_ps.append(p)

## 2초마다 process를 멈추는 generator도 넘겨줌
env.process(stop_any_process(env))

env.run(until=20)

clock 0 ticks at 2
clock 1 ticks at 2
clock 2 ticks at 2
clock 3 ticks at 2
clock 4 ticks at 2
## the clock 3 was interrupted
clock 0 ticks at 4
clock 1 ticks at 4
clock 2 ticks at 4
clock 4 ticks at 4
## the clock 0 was interrupted
clock 1 ticks at 6
clock 2 ticks at 6
clock 4 ticks at 6
## the clock 4 was interrupted
clock 1 ticks at 8
clock 2 ticks at 8
## the clock 1 was interrupted
clock 2 ticks at 10
## the clock 2 was interrupted
####################
all process was interrupted at 12


In [147]:
import simpy 
import numpy as np 

def Student(env, num, library, arrive_time):
    yield env.timeout(arrive_time)
    print("student {} arrived library at {:6.2f}".format(num, env.now))
    ## 아래와 같은 형태로 쓰면 자동으로 get, release가 된다.
    ## 단, 다른 형태로 쓸 경우에는 req = library.request(), library.release(req) 로 해주어야 함. 
    waiting_time = env.now
    with library.request() as req:
        yield req ## resource를 사용할 수 있으면 이 다음 부분이 수행됨
        waiting_time = env.now - waiting_time
        if waiting_time !=0:
            print("student {} is waiting  during {:6.2f}".format(num, waiting_time))
        study_time = np.random.triangular(left=5, right=10, mode=8)
        print("student {} start to  study at {:6.2f}".format(num, env.now))
        if library.capacity == library.count:
            print("#### library full at  {:6.2f} ####".format(env.now))
        yield env.timeout(study_time)
        print("student {} end   to  study at {:6.2f}".format(num, env.now))
        print("#### library seat available at {:6.2f} ####".format(env.now))
        
env = simpy.Environment()
library = simpy.Resource(env, capacity=2)

for i in range(0, 5):
    arrive_time = np.random.triangular(left=1, right=8, mode=3)
    stu = Student(env, i, library, arrive_time)
    print(stu)
    env.process(stu)

env.run(until=50)

<generator object Student at 0x10fa1a6d0>
<generator object Student at 0x10f9f4b48>
<generator object Student at 0x10f9f4308>
<generator object Student at 0x10f9f4d58>
<generator object Student at 0x10f9f4d00>
student 3 arrived library at   2.44
student 3 start to  study at   2.44
student 4 arrived library at   3.49
student 4 start to  study at   3.49
#### library full at    3.49 ####
student 1 arrived library at   3.71
student 2 arrived library at   3.93
student 0 arrived library at   4.11
student 3 end   to  study at  10.67
#### library seat available at  10.67 ####
student 1 is waiting  during   6.96
student 1 start to  study at  10.67
#### library full at   10.67 ####
student 4 end   to  study at  11.17
#### library seat available at  11.17 ####
student 2 is waiting  during   7.24
student 2 start to  study at  11.17
#### library full at   11.17 ####
student 1 end   to  study at  17.27
#### library seat available at  17.27 ####
student 0 is waiting  during  13.17
student 0 start to 

In [148]:
import simpy 
import numpy as np 

def Student(env, num, library, arrive_time):
    yield env.timeout(arrive_time)
    print("student {} arrived library at {:6.2f}".format(num, env.now))
    ## 아래와 같은 형태로 쓰면 자동으로 get, release가 된다.
    ## 단, 다른 형태로 쓸 경우에는 req = library.request(), library.release(req) 로 해주어야 함. 
    waiting_time = env.now
    with library.request() as req:
        if len(library.queue)!=0:
            print("{} student are already waiting for".format(len(library.queue)))
            print(library.users)
        yield req ## resource를 사용할 수 있으면 이 다음 부분이 수행됨
        waiting_time = env.now - waiting_time
        if waiting_time !=0:
            print("student {} is waiting  during {:6.2f}".format(num, waiting_time))
        study_time = np.random.triangular(left=5, right=10, mode=8)
        print("student {} start to  study at {:6.2f}".format(num, env.now))
        if library.capacity == library.count:
            print("#### library full at  {:6.2f} ####".format(env.now))
        yield env.timeout(study_time)
        print("student {} end   to  study at {:6.2f}".format(num, env.now))
        print("#### library seat available at {:6.2f} ####".format(env.now))
        
env = simpy.Environment()
library = simpy.Resource(env, capacity=2)

for i in range(0, 5):
    arrive_time = np.random.triangular(left=1, right=8, mode=3)
    stu = Student(env, i, library, arrive_time)
    print(stu)
    env.process(stu)

env.run(until=50)

<generator object Student at 0x10fa1a410>
<generator object Student at 0x10f9f4d00>
<generator object Student at 0x10fa1ab48>
<generator object Student at 0x10fa1a3b8>
<generator object Student at 0x10fa1a888>
student 4 arrived library at   1.25
student 4 start to  study at   1.25
student 3 arrived library at   2.92
student 3 start to  study at   2.92
#### library full at    2.92 ####
student 2 arrived library at   2.96
1 student are already waiting for
[<Request() object at 0x10fa10e48>, <Request() object at 0x10fa10fd0>]
student 0 arrived library at   4.45
2 student are already waiting for
[<Request() object at 0x10fa10e48>, <Request() object at 0x10fa10fd0>]
student 1 arrived library at   4.54
3 student are already waiting for
[<Request() object at 0x10fa10e48>, <Request() object at 0x10fa10fd0>]
student 4 end   to  study at   8.76
#### library seat available at   8.76 ####
student 2 is waiting  during   5.80
student 2 start to  study at   8.76
#### library full at    8.76 ####
stud

In [145]:
def aaa():
    i = 1
    while True:
        yield i 
        i+=1
a = aaa()

In [146]:
print(a)

<generator object aaa at 0x10fa1a570>


In [204]:
import numpy as np 

np.random.exponential(10)

4.692680899768591

In [ ]:
import simpy 
import numpy as np 

def customer(env, name, counter, time_in_bank):
    """
    Customer arrives ==> served ==> leaves.
    """
    arrive_time = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        ## 아래처럼 
        results = yield req | env.timeout(patience)

        wait_time = env.now - arrive_time

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait_time))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = np.random.exponential(interval)
        yield env.timeout(t)

In [223]:
import simpy 
import numpy as np 

def customer(env, name, counter, mean_service_time):
    ## counter: 사용하는 리소스 
    ## mean_service_time: 서비스 시간 평균 
    arrive_time = env.now
    print('%7.4f %s: Here I am' % (arrive_time, name))
    
    with counter.request() as req:
        yield req 
        wait_time = env.now - arrive_time
        print('%7.4f %s: Waited %6.3f' % (env.now, name, wait_time))
        service_time = np.random.exponential(mean_service_time)
        yield env.timeout(service_time)
        print('%7.4f %s: Finished' % (env.now, name))

def source(env, customer_n, interval, counter):
    ## exponential time 마다 customer를 추가해줍니다
    for i in range(customer_n):
        c = customer(env, 'Customer%02d' % i, counter, mean_service_time=5.0)
        env.process(c)
        t = np.random.exponential(interval)
        yield env.timeout(t)
        
np.random.seed(42)
env = simpy.Environment()

## 우선 counter generator를 만들어주고 
counter = simpy.Resource(env, capacity=1)
bank = source(env, 5, 3.0, counter)

env.process(bank)
env.run(until=90)


 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 1.4078 Customer01: Here I am
 5.3580 Customer02: Here I am
 8.0969 Customer03: Here I am
 8.6057 Customer04: Here I am
15.0506 Customer00: Finished
15.0506 Customer01: Waited 13.643
15.3498 Customer01: Finished
15.3498 Customer02: Waited  9.992
25.4060 Customer02: Finished
25.4060 Customer03: Waited 17.309
30.0014 Customer03: Finished
30.0014 Customer04: Waited 21.396
36.1576 Customer04: Finished


In [239]:
import simpy 
import numpy as np 

def customer(env, name, counter, mean_service_time):
    ## counter: 사용하는 리소스 
    ## mean_service_time: 서비스 시간 평균 
    arrive_time = env.now
    patience_time = np.random.uniform(2, 5)
    print('%7.4f %s: Here I am' % (arrive_time, name))
    
    with counter.request() as req:
        ## | 로 묶어주면 or  종료조건
        ## & 로 묶어주면 and 종료조건으로 인식함 
        patience_over = env.timeout(patience_time) 
        result = yield req | patience_over
        wait_time = env.now - arrive_time
        if patience_over in result:
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait_time))
        elif req in result:
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait_time))
            service_time = np.random.exponential(mean_service_time)
            yield env.timeout(service_time)
            print('%7.4f %s: Finished' % (env.now, name))

def source(env, customer_n, interval, counter):
    ## exponential time 마다 customer를 추가해줍니다
    for i in range(customer_n):
        c = customer(env, 'Customer%02d' % i, counter, mean_service_time=8.0)
        env.process(c)
        t = np.random.exponential(interval)
        yield env.timeout(t)
        
np.random.seed(42)
env = simpy.Environment()

## 우선 counter generator를 만들어주고 
counter = simpy.Resource(env, capacity=1)
bank = source(env, 5, 3.0, counter)

env.process(bank)
env.run(until=90)

 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 1.4078 Customer01: Here I am
 3.8759 Customer01: RENEGED after  2.468
 4.1466 Customer02: Here I am
 4.6554 Customer03: Here I am
 6.3209 Customer02: RENEGED after  2.174
 8.4588 Customer03: RENEGED after  3.803
10.5340 Customer00: Finished
10.6891 Customer04: Here I am
10.6891 Customer04: Waited  0.000
38.7176 Customer04: Finished


In [ ]:
import simpy 
import numpy as np 


def customer_arrivals(env, theater):
    """Create new *moviegoers* until the sim time reaches 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))

In [304]:
import simpy 
import numpy as np

## 세차머신
class CarWashMachine(object):
    ## 리소스는 아래처럼 클래스로 만들어서 처리해주는 것이 바람직할 것으로 생각됨 
    def __init__(self, env, capacity):
        self.env = env
        self.machine = simpy.Resource(self.env, capacity = capacity)
        self.users = [] ## 현재 사용중인 프로세스
        self.queue = [] ## 현재 대기열에 있는 프로세스 
    def wash(self, car_name):
        ## 아래처럼 resource 에서 이뤄지는 부분은 여기에서 작성하는 편이 더 좋을 수도 있음.
        print("{:6.2f} users: {}, queue: {}".format(env.now, self.users, self.queue))
        waiting_time = self.env.now
        with self.machine.request() as req:
            ## 바로 resource available 할 때 
            if self.machine.count < self.machine.capacity:
                self.users.append(car_name)
                yield req
            ## resource available 하지 않을 때 
            else:
                self.queue.append(car_name)
                yield req
                self.queue.remove(car_name)
                self.users.append(car_name)
            waiting_time = self.env.now - waiting_time
            if waiting_time!=0:
                print("{:6.2f} {} waited {:6.2f}".format(self.env.now, car_name, waiting_time))
            print("{:6.2f} {} wash start".format(self.env.now, car_name))    
            wash_time = np.random.exponential(30)
            yield self.env.timeout(wash_time)
            print("{:6.2f} {} wash over".format(self.env.now, car_name))
        self.users.remove(car_name)

## 도착했음을 출력하고, 리소스에 사용요청을 보내고, 사용을 끝내면 메세지츨 출력하고 종료하는 제너레이터 
def car(env, name, car_wash_machine):
    ## 도착하고, resource에 넘겨지고 그 다음을 죽 진행함. 
    print("{:6.2f} {} arrived".format(env.now, name))
    yield env.process(car_wash_machine.wash(name))
    print("{:6.2f} {} leaved".format(env.now, name))

## 랜덤으로 car를 생성해내는 제너레이터 
def source(env, car_n, car_wash_machine):
    for i in range(0, car_n):
        arrival_time = np.random.exponential(3)
        yield env.timeout(arrival_time)
        new_car = car(env, "Car{:2d}".format(i), car_wash_machine)
        env.process(new_car)

np.random.seed(42)
env = simpy.Environment()
cwm1 = CarWashMachine(env, capacity=2)

s = source(env, 10, cwm1)
env.process(s)

env.run(until=100)

  1.41 Car 0 arrived
  1.41 users: [], queue: []
  1.41 Car 0 wash start
 10.44 Car 1 arrived
 10.44 users: ['Car 0'], queue: []
 10.44 Car 1 wash start
 13.18 Car 2 arrived
 13.18 users: ['Car 0', 'Car 1'], queue: []
 13.69 Car 3 arrived
 13.69 users: ['Car 0', 'Car 1'], queue: ['Car 2']
 13.87 Car 4 arrived
 13.87 users: ['Car 0', 'Car 1'], queue: ['Car 2', 'Car 3']
 15.53 Car 1 wash over
 15.53 Car 1 leaved
 15.53 Car 2 waited   2.35
 15.53 Car 2 wash start
 19.90 Car 5 arrived
 19.90 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4']
 23.59 Car 6 arrived
 23.59 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4', 'Car 5']
 23.66 Car 7 arrived
 23.66 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4', 'Car 5', 'Car 6']
 34.17 Car 8 arrived
 34.17 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4', 'Car 5', 'Car 6', 'Car 7']
 39.53 Car 9 arrived
 39.53 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4', 'Car 5', 'Car 6', 'Car 7', 'Car 8']
 40.91 Car 0 wash over
 40.91 Car 0 leav

In [406]:
import simpy 
import numpy as np 

class Machine(object):
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False 
        
        self.process = self.env.process(self.working(repairman))
        self.env.process(self.break_machine())
        
    def working(self, repairman):
        while True:
            time_per_part = np.random.normal(20, 2)
            while True:
                start = self.env.now
                try:
                    ## 문제없이 부품을 생산한 경우 
                    yield self.env.timeout(time_per_part)
                    break
                except simpy.Interrupt:
                    ## 부품 생산중에 장비가 고장난 경우 
                    ## 이미 조립한 부분은 남기고, 추가로 조립해야 하는 시간만 계산함 
                    time_per_part -= (self.env.now - start)## remain time to part
                    ## 고장 났기 때문에 repairman에게 우선순위 높게 요청한다. 
                    self.env.process(self.repair_machine(repairman))
                    ## 고침 완료
                    ## 고쳐졌으므로 다시 고장 프로세스 가동
                    self.env.process(self.break_machine())
            self.parts_made += 1
            
    def repair_machine(self, repairman):
        with repairman.request(priority=1) as req:
            yield req ## 사용 요청 
            repair_time = np.random.uniform(30, 40)
            yield self.env.timeout(repair_time) ## 고침 
            print("{} is repaired at {:8.2f}".format(self.name, env.now))
            self.broken = False
            
    def break_machine(self):
        ## 랜덤한 시간 뒤에 현재 broken이 아니면 인터럽트를 발생시킴. 
        time_to_failure = np.random.exponential(300) 
        if self.broken == True:
            time_to_failure += np.random.uniform(30, 40)
        yield self.env.timeout(time_to_failure)
        if self.broken == False:
            print("{} is broken   at {:8.2f}".format(self.name, env.now))
            self.broken = True
            self.process.interrupt()
                
def other_jobs(env, repairman):
    ## repairman은 고치는 일 이외에도 우선순위가 낮은 다른 일들을 계속 수행하고 있음. 
    while True:
        do_time = 30.0
        while True:
            with repairman.request(priority=2) as req:
                yield req
                ## repairman의 경우 그냥 Resource가 아닌 PreemptiveResource
                ## 따라서 현재 하고 있는 일보다 priority가 높은 request가 들어왔는데 capacity가 부족할 경우, 
                ## priority가 우선인 것을 먼저 한다. 따라서 그때는 simpy.Interrupt가 발생하게 됨. 
                try:
                    start = env.now
                    yield env.timeout(do_time)
                    do_time = 0
                    break
                except simpy.Interrupt:
                    do_time -= (env.now - start)
        
####################
np.random.seed(42)
print("Machine shop")

env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)

machines = [Machine(env, 'Machine %d' % i, repairman) for i in range(0, 10)]
env.process(other_jobs(env, repairman))
env.run(until=60*5)

print("#"*31)
print('Machine shop results after %s weeks' % 4)
sum_parts = 0
for machine in machines:
    print('%s made %d parts.' % (machine.name, machine.parts_made))
    sum_parts += machine.parts_made
print("#"*31)
print("{} parts produced".format(sum_parts))

Machine shop
Machine 4 is broken   at     6.24
Machine 2 is broken   at    17.95
Machine 4 is repaired at    39.16
Machine 6 is broken   at    60.20
Machine 7 is broken   at    60.78
Machine 2 is repaired at    74.11
Machine 9 is broken   at   103.27
Machine 6 is repaired at   107.42
Machine 7 is repaired at   138.50
Machine 7 is broken   at   144.37
Machine 8 is broken   at   169.66
Machine 9 is repaired at   176.58
Machine 6 is broken   at   194.57
Machine 7 is repaired at   211.60
Machine 8 is repaired at   250.97
Machine 1 is broken   at   273.88
Machine 6 is repaired at   287.89
###############################
Machine shop results after 4 weeks
Machine 0 made 14 parts.
Machine 1 made 14 parts.
Machine 2 made 14 parts.
Machine 3 made 16 parts.
Machine 4 made 14 parts.
Machine 5 made 14 parts.
Machine 6 made 15 parts.
Machine 7 made 14 parts.
Machine 8 made 15 parts.
Machine 9 made 15 parts.
###############################
145 parts produced


In [ ]:
import simpy 
import numpy as np 

class Machine(object):
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False 
        
        # Start "working" and "break_machine" processes for this machine.
        self.process = self.env.process(self.working(repairman))
        self.env.process(self.break_machine())
        
    def working(self, repairman):
        while True:
            time_per_part = np.random.normal(20, 2)
            while True:
                start = self.env.now
                try:
                    ## 문제없이 부품을 생산한 경우 
                    yield self.env.timeout(time_per_part)
                    break
                except simpy.Interrupt:
                    ## 부품 생산중에 장비가 고장난 경우 
                    ## 이미 조립한 부분은 남기고, 추가로 조립해야 하는 시간만 계산함 
                    time_per_part -= (self.env.now - start)## remain time to part
                    ## 고장 났기 때문에 repairman에게 우선순위 높게 요청한다. 
                    with repairman.request(priority=1) as req:
                        yield req ## 사용 요청 
                        repair_time = np.random.uniform(30, 40)
                        yield self.env.timeout(repair_time) ## 고침 
                        #print("{} is repaired at {:8.2f}".format(self.name, env.now))
                    ## 고침 완료
                    self.broken = False
                    ## 고쳐졌으므로 다시 고장 프로세스 가동
                    self.env.process(self.break_machine())
            self.parts_made += 1
    def repair_machine(self, repairman):
        
    def break_machine(self):
        ## 랜덤한 시간 뒤에 현재 broken이 아니면 인터럽트를 발생시킴. 
        time_to_failure = np.random.exponential(300) 
        if self.broken == True:
            time_to_failure += np.random.uniform(30, 40)
        yield self.env.timeout(time_to_failure)
        if self.broken == False:
            #print("{} is broken   at {:8.2f}".format(self.name, env.now))
            self.broken = True
            self.process.interrupt()
                
def other_jobs(env, repairman):
    while True:
        do_time = 30.0
        while True:
            with repairman.request(priority=2) as req:
                ## repairman의 경우 그냥 Resource가 아닌 PreemptiveResource
                ## 따라서 현재 하고 있는 일보다 priority가 높은 request가 들어왔는데 capacity가 부족할 경우, 
                ## priority가 우선인 것을 먼저 한다. 따라서 그때는 simpy.Interrupt가 발생하게 됨. 
                yield req
                try:
                    start = env.now
                    yield env.timeout(do_time)
                    do_time = 0
                    break
                except simpy.Interrupt:
                    do_time -= (env.now - start)
        

np.random.seed(42)
print("Machine shop")

env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)

machines = [Machine(env, 'Machine %d' % i, repairman) for i in range(0, 10)]
env.process(other_jobs(env, repairman))
env.run(until=60*3)
print("#"*31)
print('Machine shop results after %s weeks' % 4)
sum_parts = 0
for machine in machines:
    print('%s made %d parts.' % (machine.name, machine.parts_made))
    sum_parts += machine.parts_made
print("#"*31)
print("{} parts produced".format(sum_parts))